# Patch daily with jura

In [1]:
import os
import numpy as np
from astropy.table import Table, join
from astropy.io import fits
from specprodDB.util import cameraid

In [2]:
daily_tiles_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'daily', 'tiles-daily.csv')
daily_exposures_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'daily', 'exposures-daily.fits')
jura_tiles_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'jura', 'tiles-jura.csv')
jura_exposures_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'jura', 'exposures-jura.fits')
daily_tiles = Table.read(daily_tiles_file, format='ascii.csv')
daily_exposures = Table.read(daily_exposures_file, format='fits', hdu='EXPOSURES')
daily_frames = Table.read(daily_exposures_file, format='fits', hdu='FRAMES')
jura_tiles = Table.read(jura_tiles_file, format='ascii.csv')
jura_exposures = Table.read(jura_exposures_file, format='fits', hdu='EXPOSURES')
jura_frames = Table.read(jura_exposures_file, format='fits', hdu='FRAMES')

## Find jura exposures not in daily, daily exposures not in jura

In [3]:
assert (np.unique(jura_exposures['EXPID']) == sorted(jura_exposures['EXPID'])).all()
assert (np.unique(daily_exposures['EXPID']) == sorted(daily_exposures['EXPID'])).all()
assert (np.unique(jura_frames['EXPID']) == sorted(jura_exposures['EXPID'])).all()
assert (np.unique(daily_frames['EXPID']) == sorted(daily_exposures['EXPID'])).all()

In [4]:
first_jura_exposure, last_jura_exposure = jura_exposures['EXPID'].min(), jura_exposures['EXPID'].max()

In [5]:
jura_expid_set = frozenset(jura_exposures['EXPID'].tolist())
daily_expid_set = frozenset(daily_exposures['EXPID'].tolist())

In [6]:
jura_not_in_daily = jura_expid_set - daily_expid_set
jura_not_in_daily

frozenset({80478, 80681, 80688, 80691, 82603, 82622, 82625, 221977})

In [7]:
daily_not_in_jura = daily_expid_set - jura_expid_set
# daily_not_in_jura

## Patch frames

In [8]:
daily_frames_join = Table()
daily_frames_join['FRAMEID'] = np.array([100*row['EXPID'] + cameraid(row['CAMERA']) for row in daily_frames])
daily_frames_join['DAILY_INDEX'] = np.arange(len(daily_frames))

In [9]:
jura_frames_join = Table()
jura_frames_join['FRAMEID'] = np.array([100*row['EXPID'] + cameraid(row['CAMERA']) for row in jura_frames])
jura_frames_join['JURA_INDEX'] = np.arange(len(jura_frames))

In [10]:
j = join(daily_frames_join, jura_frames_join, join_type='outer', keys='FRAMEID')
j

FRAMEID,DAILY_INDEX,JURA_INDEX
int64,int64,int64
5098600,567237,--
5098601,567238,--
5098602,567239,--
5098603,567240,--
5098604,567241,--
5098605,567242,--
5098606,567243,--
5098607,567244,--
5098608,567245,--


In [11]:
daily_frames_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['DAILY_INDEX']
jura_frames_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['JURA_INDEX']

In [12]:
daily_frames_patched = daily_frames.copy()
for column in jura_frames.colnames:
    # jura does not have TSNR2_ALPHA
    daily_frames_patched[column][daily_frames_index] = jura_frames[column][jura_frames_index]
daily_frames_patched

NIGHT,EXPID,TILEID,EXPTIME,SEEING_ETC,EFFTIME_ETC,CAMERA,TSNR2_ELG,TSNR2_BGS,TSNR2_QSO,TSNR2_LRG,SURVEY,GOALTYPE,FAFLAVOR,MINTFRAC,GOALTIME,TSNR2_LYA,TSNR2_ALPHA,FAPRGRM,AIRMASS,EBV,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,TILERA,TILEDEC,MJD
int32,int32,int32,float32,float64,float64,bytes2,float64,float64,float64,float64,bytes7,bytes7,bytes19,float64,float64,float64,float64,bytes19,float32,float32,float64,float64,float64,float64,float64,float64
20210916,100469,82130,180.0338,0.0,-99.0,b0,0.0,0.0,0.0,0.0,cmx,unknown,dithprec,0.9,-99.0,0.0,1e+20,dithprec,0.0,0.0,0.0,0.0,0.0,336.0,30.0,59474.238451961
20210916,100469,82130,180.0338,0.0,-99.0,b1,0.0,0.0,0.0,0.0,cmx,unknown,dithprec,0.9,-99.0,0.0,1e+20,dithprec,0.0,0.0,0.0,0.0,0.0,336.0,30.0,59474.238451961
20210916,100469,82130,180.0338,0.0,-99.0,b2,0.0,0.0,0.0,0.0,cmx,unknown,dithprec,0.9,-99.0,0.0,1e+20,dithprec,0.0,0.0,0.0,0.0,0.0,336.0,30.0,59474.238451961
20210916,100469,82130,180.0338,0.0,-99.0,b3,0.0,0.0,0.0,0.0,cmx,unknown,dithprec,0.9,-99.0,0.0,1e+20,dithprec,0.0,0.0,0.0,0.0,0.0,336.0,30.0,59474.238451961
20210916,100469,82130,180.0338,0.0,-99.0,b4,0.0,0.0,0.0,0.0,cmx,unknown,dithprec,0.9,-99.0,0.0,1e+20,dithprec,0.0,0.0,0.0,0.0,0.0,336.0,30.0,59474.238451961
20210916,100469,82130,180.0338,0.0,-99.0,b5,0.0,0.0,0.0,0.0,cmx,unknown,dithprec,0.9,-99.0,0.0,1e+20,dithprec,0.0,0.0,0.0,0.0,0.0,336.0,30.0,59474.238451961
20210916,100469,82130,180.0338,0.0,-99.0,b6,0.0,0.0,0.0,0.0,cmx,unknown,dithprec,0.9,-99.0,0.0,1e+20,dithprec,0.0,0.0,0.0,0.0,0.0,336.0,30.0,59474.238451961
20210916,100469,82130,180.0338,0.0,-99.0,b7,0.0,0.0,0.0,0.0,cmx,unknown,dithprec,0.9,-99.0,0.0,1e+20,dithprec,0.0,0.0,0.0,0.0,0.0,336.0,30.0,59474.238451961
20210916,100469,82130,180.0338,0.0,-99.0,b8,0.0,0.0,0.0,0.0,cmx,unknown,dithprec,0.9,-99.0,0.0,1e+20,dithprec,0.0,0.0,0.0,0.0,0.0,336.0,30.0,59474.238451961


## Patch exposures

In [13]:
daily_exposures_join = Table()
daily_exposures_join['EXPID'] = daily_exposures['EXPID']
daily_exposures_join['DAILY_INDEX'] = np.arange(len(daily_exposures))

In [14]:
jura_exposures_join = Table()
jura_exposures_join['EXPID'] = jura_exposures['EXPID']
jura_exposures_join['JURA_INDEX'] = np.arange(len(jura_exposures))

In [15]:
j = join(daily_exposures_join, jura_exposures_join, join_type='outer', keys='EXPID')
j

EXPID,DAILY_INDEX,JURA_INDEX
int32,int64,int64
50986,0,--
50988,1,--
50991,2,--
50995,3,--
51001,4,--
51002,5,--
51028,6,--
51029,7,--
51030,8,--


In [16]:
daily_exposures_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['DAILY_INDEX']
jura_exposures_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['JURA_INDEX']

In [17]:
daily_exposures_patched = daily_exposures.copy()
for column in jura_exposures.colnames:
    daily_exposures_patched[column][daily_exposures_index] = jura_exposures[column][jura_exposures_index]
daily_exposures_patched

NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes19,bytes19,float64,float64,float64,bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
20200219,50986,63075,0.0,0.0,0.0,unknown,other,unknown,unknown,60.0,0.0,1000.0,unknown,0.9,1.043362,0.0942080095410347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20200219,50988,70004,0.0,0.0,0.0,unknown,other,unknown,unknown,900.0,680.3945587158206,1000.0,unknown,0.9,1.104283,0.04696561396121979,0.0,0.0,79.11564636230469,19.950475692749023,54.666229248046875,57.56499099731445,5267.74755859375,0.0,0.0,0.0,664.2277979694464,680.3945587158206,737.484658203125,658.5295680370829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.955594041629173,21.284226418609467,19.790755796726657,0.0,0.0,0.0,0.0
20200219,50991,63075,0.0,0.0,0.0,unknown,other,unknown,unknown,60.0,0.0,1000.0,unknown,0.9,1.008076,0.0942080095410347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20200219,50995,70004,0.0,0.0,0.0,unknown,other,unknown,unknown,900.0,846.9120193481449,1000.0,unknown,0.9,1.034227,0.04696561396121979,0.0,0.0,98.47814178466797,24.324670791625977,67.42870330810547,66.30667877197266,6406.125,0.0,0.0,0.0,819.299588326338,846.9120193481449,896.8575000000001,758.5323609573368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.966302914201346,21.262359707846763,19.752054491279942,0.0,0.0,0.0,0.0
20200219,51001,70004,0.0,0.0,0.0,unknown,other,unknown,unknown,900.0,999.3838150024418,1000.0,unknown,0.9,1.022479,0.04696561396121979,0.0,0.0,116.2074203491211,28.303956985473633,77.67794036865234,71.61495208740234,7325.822265625,0.0,0.0,0.0,943.8340268130958,999.3838150024418,1025.6151171875001,819.2577232456027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.00288157116244,21.356993054425978,19.844414121595623,0.0,0.0,0.0,0.0
20200219,51002,70004,0.0,0.0,0.0,unknown,other,unknown,unknown,900.0,1093.0981079101566,1000.0,unknown,0.9,1.034986,0.04696561396121979,0.0,0.0,127.10443115234375,30.669572830200195,85.87296295166016,71.7078628540039,8028.75,0.0,0.0,0.0,1043.4085151122983,1093.0981079101566,1124.025,820.3205999339526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.021436962721946,21.35818612569135,19.888063776015954,0.0,0.0,0.0,0.0
20200219,51028,70005,0.0,0.0,0.0,unknown,other,unknown,unknown,0.0,0.0,1000.0,unknown,0.9,1.02954,0.018338840454816818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20200219,51029,70005,0.0,0.0,0.0,unknown,other,unknown,unknown,0.0,0.0,1000.0,unknown,0.9,1.025211,0.018338840454816818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20200219,51030,70005,0.0,0.0,0.0,unknown,other,unknown,unknown,900.0,1343.9888305664067,1000.0,unknown,0.9,1.021381,0.018338840454816818,0.0,0.0,156.27777099609375,38.330753326416016,103

## Patch tiles

In [18]:
assert (np.unique(jura_exposures['TILEID']) == sorted(jura_tiles['TILEID'])).all()
assert (np.unique(daily_exposures['TILEID']) == sorted(daily_tiles['TILEID'])).all()

In [19]:
daily_tiles_join = Table()
daily_tiles_join['TILEID'] = daily_tiles['TILEID']
daily_tiles_join['DAILY_INDEX'] = np.arange(len(daily_tiles))

In [20]:
jura_tiles_join = Table()
jura_tiles_join['TILEID'] = jura_tiles['TILEID']
jura_tiles_join['JURA_INDEX'] = np.arange(len(jura_tiles))

In [21]:
j = join(daily_tiles_join, jura_tiles_join, join_type='outer', keys='TILEID')
j

TILEID,DAILY_INDEX,JURA_INDEX
int64,int64,int64
1,432,202
2,460,230
3,476,267
4,508,292
5,525,321
6,611,400
7,736,476
8,752,486
9,778,513


In [22]:
# Apparently every tile in daily also appears in jura, so j['DAILY_INDEX'] doesn't need to be masked.
daily_tiles_index = j[(~j['JURA_INDEX'].mask)]['DAILY_INDEX']
jura_tiles_index = j[(~j['JURA_INDEX'].mask)]['JURA_INDEX']

In [23]:
daily_tiles_patched = daily_tiles.copy()
for column in jura_tiles.colnames:
    daily_tiles_patched[column][daily_tiles_index] = jura_tiles[column][jura_tiles_index]
daily_tiles_patched

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str7,float64,int64
70004,unknown,--,unknown,unknown,4,3600.0,116.0,20.7,0.0,3619.8,0.0,1000.0,obsend,3470.8,3619.8,3784.0,3056.6,unknown,0.9,20200219
70508,unknown,--,unknown,unknown,6,1800.0,133.4125,11.6818,0.0,108.5,0.0,1000.0,obsstart,93.6,108.5,95.7,50.6,unknown,0.9,20200225
70506,unknown,--,unknown,unknown,3,900.0,133.4125,11.6818,0.0,61.6,0.0,1000.0,obsstart,53.6,61.6,54.9,33.7,unknown,0.9,20200225
70512,unknown,--,unknown,unknown,7,3150.0,132.85,12.32,0.0,389.8,0.0,1000.0,obsstart,351.9,389.8,364.7,227.2,unknown,0.9,20200226
70514,unknown,--,unknown,unknown,16,1680.0,132.85,12.32,0.0,118.3,0.0,1000.0,obsstart,112.0,118.3,120.2,74.4,unknown,0.9,20200227
70502,unknown,--,unknown,unknown,41,8415.0,180.0,-0.5,0.0,358.2,0.0,1000.0,obsstart,317.9,358.2,325.0,260.2,unknown,0.9,20200227
70513,unknown,--,unknown,unknown,13,2220.0,133.42,11.65,0.0,3.7,0.0,1000.0,obsstart,3.4,3.7,3.5,2.5,unknown,0.9,20200229
70500,unknown,--,unknown,unknown,14,7000.0,119.0,50.0,0.0,1389.0,0.0,1000.0,obsend,1145.4,1389.0,1160.5,770.4,unknown,0.9,20200303
70005,unknown,--,unknown,unknown,17,13500.0,158.0,25.0,0.0,8463.1,0.0,1000.0,obsend,8088.5,8463.1,8858.6,9174.7,unknown,0.9,20200303


## Write out the patched files

In [24]:
daily_tiles_patched.write(os.path.join(os.environ['SCRATCH'], 'tiles-daily-patched-with-jura.csv'), format='ascii.csv', overwrite=True)

In [25]:
daily_exposures_patched.write(os.path.join(os.environ['SCRATCH'], 'exposures-daily-patched-with-jura.csv'), format='ascii.csv', overwrite=True)

In [26]:
daily_exposures_fits = fits.HDUList([fits.PrimaryHDU(), fits.table_to_hdu(daily_exposures_patched), fits.table_to_hdu(daily_frames_patched)])

In [27]:
daily_exposures_fits.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  EXPOSURES     1 BinTableHDU    111   23585R x 51C   ['J', 'J', 'J', 'D', 'D', 'D', '7A', '6A', '19A', '19A', 'D', 'D', 'D', '7A', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D']   
  2  FRAMES        1 BinTableHDU     63   694207R x 27C   ['J', 'J', 'J', 'E', 'D', 'D', '2A', 'D', 'D', 'D', 'D', '7A', '7A', '19A', 'D', 'D', 'D', 'D', '19A', 'E', 'E', 'D', 'D', 'D', 'D', 'D', 'D']   


In [28]:
daily_exposures_fits.writeto(os.path.join(os.environ['SCRATCH'], 'exposures-daily-patched-with-jura.fits'), overwrite=True)